In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample

import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
import sklearn
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,Concatenate,concatenate, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
#from tensorflow.keras.utils import np_utils
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.regularizers import l2

In [2]:
tensorflow.keras.backend.clear_session()

In [3]:
model_ecg = Sequential()
model_ecg.add(BatchNormalization(input_shape=(1408,1)))
model_ecg.add(Conv1D(3, kernel_size=(100),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Conv1D(50, (10)))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(Conv1D(30, (30)))
model_ecg.add(MaxPooling1D(pool_size=(2)))
model_ecg.add(Activation("relu"))
model_ecg.add(BatchNormalization())
model_ecg.add(Flatten())
model_ecg.add(Dropout(0.25))
model_ecg.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_ecg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1408, 1)           4         
_________________________________________________________________
conv1d (Conv1D)              (None, 655, 3)            303       
_________________________________________________________________
activation (Activation)      (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 327, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 159, 50)           0         
_________________________________________________________________
activation_1 (Activation)    (None, 159, 50)           0

In [4]:
model_ecg.load_weights('./best_weights_ecg_32layer.hdf5')

In [5]:
weight_values=model_ecg.get_weights()

In [6]:
first_input = Input(shape=(1408,1))
first_batch = BatchNormalization(weights=weight_values[0:4],trainable=False)(first_input)
first_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_values[4],weight_values[5]],trainable=False)(first_batch)
first_act=Activation("relu")(first_conv)
first_maxpool= MaxPooling1D(pool_size=(2),strides=2)(first_act)
first_conv2=Conv1D(50,kernel_size=(10),weights=[weight_values[6],weight_values[7]],trainable=False)(first_maxpool)
first_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(first_conv2)
first_act2=Activation("relu")(first_maxpool2)
first_conv3=Conv1D(30,kernel_size=(30),weights=[weight_values[8],weight_values[9]],trainable=False)(first_act2)
first_maxpool3=MaxPooling1D(pool_size=(2))(first_conv3)
first_act3=Activation("relu")(first_maxpool3)
first_batch2 = BatchNormalization(weights=weight_values[10:14],trainable=False)(first_act3)
first_flatten = Flatten()(first_batch2)
first_dropout=Dropout(0.33)(first_flatten)
first_final=Flatten()(first_dropout)

In [7]:
model_spo2 = Sequential()
model_spo2.add(BatchNormalization(input_shape=(88,1)))
model_spo2.add(Conv1D(6, kernel_size=(25),padding='same'))
model_spo2.add(Activation("relu"))
#model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Conv1D(50, (10),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(Conv1D(30, (15),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(BatchNormalization())
model_spo2.add(Flatten())
model_spo2.add(Dropout(0.25))
model_spo2.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_spo2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 88, 1)             4         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 88, 6)             156       
_________________________________________________________________
activation_6 (Activation)    (None, 88, 6)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 88, 50)            3050      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 44, 50)            0         
_________________________________________________________________
activation_7 (Activation)    (None, 44, 50)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 44, 30)           

In [8]:
model_spo2.load_weights('./best_weights_spo2_3_312layer.hdf5')

In [9]:
weight_spo2=model_spo2.get_weights()

In [10]:
second_input = Input(shape=(88,1))
second_batch = BatchNormalization(weights=weight_spo2[0:4],trainable=False)(second_input)
second_conv= Conv1D(6,kernel_size=(25),padding='same',weights=[weight_spo2[4],weight_spo2[5]],trainable=False)(second_batch)
second_act=Activation("relu")(second_conv)
second_conv2=Conv1D(50,kernel_size=(10),padding='same',weights=[weight_spo2[6],weight_spo2[7]],trainable=False)(second_act)
second_maxpool2=MaxPooling1D(pool_size=(2))(second_conv2)
second_act2=Activation("relu")(second_maxpool2)
second_conv3=Conv1D(30,kernel_size=(15),padding='same',weights=[weight_spo2[8],weight_spo2[9]],trainable=False)(second_maxpool2)
second_maxpool3=MaxPooling1D(pool_size=(2))(second_conv3)
second_act3=Activation("relu")(second_maxpool3)
second_batch2 = BatchNormalization(weights=weight_spo2[10:14],trainable=False)(second_act3)
second_final = Flatten()(second_batch2)


In [11]:
model_abdo = Sequential()
model_abdo.add(BatchNormalization(input_shape=(1408,1)))
model_abdo.add(Conv1D(3, kernel_size=(100),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Conv1D(50, (10)))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(Conv1D(30, (30)))
model_abdo.add(MaxPooling1D(pool_size=(2)))
model_abdo.add(Activation("relu"))
model_abdo.add(BatchNormalization())
model_abdo.add(Flatten())
model_abdo.add(Dropout(0.25))
model_abdo.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_abdo.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 1408, 1)           4         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 655, 3)            303       
_________________________________________________________________
activation_12 (Activation)   (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 327, 3)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 159, 50)           0         
_________________________________________________________________
activation_13 (Activation)   (None, 159, 50)          

In [12]:
model_abdo.load_weights('./best_weights_abdo_32layer.hdf5')

In [13]:
weight_abdo=model_abdo.get_weights()

In [14]:
third_input = Input(shape=(1408,1))
third_batch = BatchNormalization(weights=weight_abdo[0:4],trainable=False)(third_input)
third_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_abdo[4],weight_abdo[5]],trainable=False)(third_batch)
third_act=Activation("relu")(third_conv)
third_maxpool= MaxPooling1D(pool_size=(2),strides=2)(third_act)
third_conv2=Conv1D(50,kernel_size=(10),weights=[weight_abdo[6],weight_abdo[7]],trainable=False)(third_maxpool)
third_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(third_conv2)
third_act2=Activation("relu")(third_maxpool2)
third_conv3=Conv1D(30,kernel_size=(30),weights=[weight_abdo[8],weight_abdo[9]],trainable=False)(third_act2)
third_maxpool3=MaxPooling1D(pool_size=(2))(third_conv3)
third_act3=Activation("relu")(third_maxpool3)
third_batch2 = BatchNormalization(weights=weight_abdo[10:14],trainable=False)(third_act3)
third_flatten = Flatten()(third_batch2)
third_dropout=Dropout(0.33)(third_flatten)
third_final=Flatten()(third_dropout)

In [15]:
merge_layer=concatenate([first_final, second_final, third_final])

In [16]:
merge_batch1 = BatchNormalization()(merge_layer)
merge_drop2 = Dropout(0.25)(merge_batch1)
merge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)
merge_output= Activation("softmax")(merge_dense2)


In [17]:
model_merge = Model(inputs=[first_input, second_input, third_input], outputs=merge_output)

In [18]:
model_merge.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 1408, 1)      4           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 1408, 1)      4           input_3[0][0]                    
______________________________________________________________________________________________

In [19]:
batch_size = 64
epochs = 50

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model_merge.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy',
                      tensorflow.keras.metrics.Precision(class_id=1,name='prec'),
                    tensorflow.keras.metrics.Recall(class_id=1,name='rec')])

In [20]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',




valid_features1=np.zeros((1,1408))
valid_features2=np.zeros((1,1408))
valid_features3=np.zeros((1,1408))
test_features1=np.zeros((1,1408))
test_features2=np.zeros((1,1408))
test_features3=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        spo2_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_valid.mat')
        spo2_valid = np.array(spo2_valid['spo2_valid'])
        ecg_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_valid.mat')
        ecg_valid = np.array(ecg_valid['ecg_valid'])
        abdo_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_valid.mat')
        abdo_valid = np.array(abdo_valid['abdo_valid'])
        spo2_valid_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_valid_labels.mat')
        spo2_valid_labels = np.array(spo2_valid_labels['class_valid'])
        valid_features1=np.append(valid_features1,spo2_valid,axis=0)
        valid_features2=np.append(valid_features2,ecg_valid,axis=0)
        valid_features3=np.append(valid_features3,abdo_valid,axis=0)
        valid_labels=np.append(valid_labels,spo2_valid_labels)
       
        
        spo2_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_test.mat')
        spo2_test = np.array(spo2_test['spo2_test'])
        ecg_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_test.mat')
        ecg_test = np.array(ecg_test['ecg_test'])
        abdo_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_test.mat')
        abdo_test = np.array(abdo_test['abdo_test'])
        spo2_test_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_test_labels.mat')
        spo2_test_labels = np.array(spo2_test_labels['class_test'])
        test_features1=np.append(test_features1,spo2_test,axis=0)
        test_features2=np.append(test_features2,ecg_test,axis=0)
        test_features3=np.append(test_features3,abdo_test,axis=0)
        test_labels=np.append(test_labels,spo2_test_labels)
        
spo2_valid=valid_features1[1:,0::16]
ecg_valid=valid_features2[1:]
abdo_valid=valid_features3[1:]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()

spo2_test=test_features1[1:,0::16]
ecg_test=test_features2[1:]
abdo_test=test_features3[1:]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [21]:
train_features1=np.zeros((1,1408))
train_features2=np.zeros((1,1408))
train_features3=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        spo2_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_train.mat')
        spo2_train = np.array(spo2_train['spo2_train'])
        ecg_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_train.mat')
        ecg_train = np.array(ecg_train['ecg_train'])
        abdo_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_train.mat')
        abdo_train = np.array(abdo_train['abdo_train'])
        spo2_train_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_train_labels.mat')
        spo2_train_labels = np.array(spo2_train_labels['class_train'])
        train_features1=np.append(train_features1,spo2_train,axis=0)
        train_features2=np.append(train_features2,ecg_train,axis=0)
        train_features3=np.append(train_features3,abdo_train,axis=0)
        train_labels=np.append(train_labels,spo2_train_labels)
        
spo2_train=train_features1[1:,0::16]
ecg_train=train_features2[1:]
abdo_train=train_features3[1:]
train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [22]:
train_features1=0
train_features2=0
train_features3=0
valid_features1=0
valid_features2=0
valid_features3=0
test_features1=0
test_features2=0
test_features3=0

In [23]:
ecg_train_mean=np.mean(ecg_train)
ecg_train_std=np.std(ecg_train)

In [24]:
for i in range(ecg_train.shape[0]):
    ecg_train[i,:]=(ecg_train[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_valid.shape[0]):
    ecg_valid[i,:]=(ecg_valid[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_test.shape[0]):
    ecg_test[i,:]=(ecg_test[i,:]-ecg_train_mean)/ecg_train_std
    

ecg_train=np.expand_dims(ecg_train, axis=2)
ecg_valid=np.expand_dims(ecg_valid, axis=2)
ecg_test=np.expand_dims(ecg_test, axis=2)

In [25]:
abdo_train_mean=np.mean(abdo_train)
abdo_train_std=np.std(abdo_train)

In [26]:
for i in range(abdo_train.shape[0]):
    abdo_train[i,:]=(abdo_train[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_valid.shape[0]):
    abdo_valid[i,:]=(abdo_valid[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_test.shape[0]):
    abdo_test[i,:]=(abdo_test[i,:]-abdo_train_mean)/abdo_train_std
    

abdo_train=np.expand_dims(abdo_train, axis=2)
abdo_valid=np.expand_dims(abdo_valid, axis=2)
abdo_test=np.expand_dims(abdo_test, axis=2)

In [27]:
spo2_train_mean=np.mean(spo2_train)
spo2_train_std=np.std(spo2_train)

In [28]:
for i in range(spo2_train.shape[0]):
    spo2_train[i,:]=(spo2_train[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_valid.shape[0]):
    spo2_valid[i,:]=(spo2_valid[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_test.shape[0]):
    spo2_test[i,:]=(spo2_test[i,:]-spo2_train_mean)/spo2_train_std

spo2_train=np.expand_dims(spo2_train, axis=2)
spo2_valid=np.expand_dims(spo2_valid, axis=2)
spo2_test=np.expand_dims(spo2_test, axis=2)

In [29]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [30]:
num_classes=2
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [32]:
best_weights_filepath_merge = './best_weights_fusion_abdo_17_8.hdf5'
mcp_merge = ModelCheckpoint(best_weights_filepath_merge, monitor="val_accuracy",
                      save_best_only=True, save_weights_only=False)
history = model_merge.fit([ecg_train,spo2_train, abdo_train], y_train_wide,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=([ecg_valid,spo2_valid, abdo_valid], y_valid_wide),
                         callbacks=[mcp_merge])

Epoch 1/50
12593/12593 [==============================] - 136s 11ms/step - loss: 0.0211 - accuracy: 0.9944 - prec: 0.9936 - rec: 0.9951 - val_loss: 0.0192 - val_accuracy: 0.9958 - val_prec: 0.9974 - val_rec: 0.9941
Epoch 2/50
12593/12593 [==============================] - 104s 8ms/step - loss: 0.0216 - accuracy: 0.9942 - prec: 0.9934 - rec: 0.9949 - val_loss: 0.0187 - val_accuracy: 0.9952 - val_prec: 0.9975 - val_rec: 0.9928
Epoch 3/50
12593/12593 [==============================] - 98s 8ms/step - loss: 0.0212 - accuracy: 0.9943 - prec: 0.9936 - rec: 0.9950 - val_loss: 0.0220 - val_accuracy: 0.9953 - val_prec: 0.9975 - val_rec: 0.9930
Epoch 4/50
12593/12593 [==============================] - 97s 8ms/step - loss: 0.0215 - accuracy: 0.9941 - prec: 0.9934 - rec: 0.9949 - val_loss: 0.0197 - val_accuracy: 0.9959 - val_prec: 0.9973 - val_rec: 0.9946
Epoch 5/50
12593/12593 [==============================] - 98s 8ms/step - loss: 0.0214 - accuracy: 0.9943 - prec: 0.9935 - rec: 0.9950 - val_loss:

12593/12593 [==============================] - 97s 8ms/step - loss: 0.0210 - accuracy: 0.9942 - prec: 0.9935 - rec: 0.9949 - val_loss: 0.0182 - val_accuracy: 0.9958 - val_prec: 0.9974 - val_rec: 0.9941
Epoch 40/50
12593/12593 [==============================] - 97s 8ms/step - loss: 0.0211 - accuracy: 0.9942 - prec: 0.9935 - rec: 0.9949 - val_loss: 0.0204 - val_accuracy: 0.9953 - val_prec: 0.9974 - val_rec: 0.9932
Epoch 41/50
12593/12593 [==============================] - 97s 8ms/step - loss: 0.0213 - accuracy: 0.9941 - prec: 0.9934 - rec: 0.9948 - val_loss: 0.0214 - val_accuracy: 0.9957 - val_prec: 0.9976 - val_rec: 0.9938
Epoch 42/50
12593/12593 [==============================] - 97s 8ms/step - loss: 0.0213 - accuracy: 0.9942 - prec: 0.9934 - rec: 0.9950 - val_loss: 0.0212 - val_accuracy: 0.9957 - val_prec: 0.9974 - val_rec: 0.9940
Epoch 43/50
12593/12593 [==============================] - 98s 8ms/step - loss: 0.0212 - accuracy: 0.9941 - prec: 0.9934 - rec: 0.9949 - val_loss: 0.0182 - 

# load weights

In [39]:
model_merge.load_weights('./best_weights_fusion_abdo_17_8.hdf5')
y_pred = model_merge.predict([ecg_test,spo2_test, abdo_test])
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[50237,   131],
       [   14,  1354]], dtype=int64)

In [40]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9971973094170403

In [41]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.9897660818713451

In [42]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9973991423125794